## 词袋模型

词袋模型(Bag of Words,简称BoW)，所谓的词袋模型是一种用机器学习算法对文本进行建模时表示文本数据的方法。
词袋模型假设我们不考虑文本中词与词之间的上下文关系，仅仅只考虑所有词的权重。而权重与词在文本中出现的频率有关。

词袋模型首先会进行分词，在分词之后，通过统计每个词在文本中出现的次数，我们就可以得到该文本基于词的特征，如果将各个文本样本的这些词与对应的词频放在一起，就是我们常说的向量化。向量化完毕后一般也会使用TF-IDF进行特征的权重修正，再将特征进行标准化。 再进行一些其他的特征工程后，就可以将数据带入机器学习算法进行分类聚类了。

## Word2Vec

word2vec vs one-hot:

one-hot representation:
优点：简单，在过去NLP任务中一直处于霸主地位；
缺点：特征向量维度高，容易引发维度灾难；词的语义相似度无法通过词向量的计算进行体现，不太适合深度学习的相关任务要求；
distributed representation:
优点：向量维度低且稠密，信息密度高；词间的相似度可以通过词向量的计算体现；
缺点：学习过程相对复杂且受训练语料的影响很大。

向量空间模型：向量空间模型 (VSMs)将词汇表达（嵌套）于一个连续的向量空间中，语义近似的词汇被映射为相邻的数据点。向量空间模型在自然语言处理领域中有着漫长且丰富的历史，不过几乎所有利用这一模型的方法都依赖于分布式假设，其核心思想为出现于上下文情景中的词汇都有相类似的语义。采用这一假设的研究方法大致分为以下两类：基于计数的方法 (e.g. 潜在语义分析， Glove)， 和 预测方法 (e.g. 神经概率化语言模型，word2vec).

即：基于计数的方法计算某词汇与其邻近词汇在一个大型语料库中共同出现的频率及其他统计量，然后将这些统计量映射到一个小型且稠密的向量中。预测方法则试图直接从某词汇的邻近词汇对其进行预测，在此过程中利用已经学习到的小型且稠密的嵌套向量。

![word2vec](https://ws1.sinaimg.cn/large/006tKfTcly1g1171ko023j30go09bt95.jpg)

具体步骤：
1、在输入层，一个词被转化为One-Hot向量。
2、然后在第一个隐层，输入的是一个 wx*b（x就是输入的词向量, w、b是参数），做一个线性模型,注意已这里只是简单的映射，并没有非线性激活函数,当然一个神经元可以是线性的，这时就相当于一个线性回归函数。
3、第三层可以简单看成一个分类器，用的是Softmax回归，最后输出的是每个词对应的概率。


In [3]:
import jieba
import warnings
warnings.filterwarnings("ignore")

with open('./tianlongbabu.txt', errors='ignore', encoding='utf-8') as fp:
   lines = fp.readlines()
   for line in lines:
       seg_list = jieba.cut(line)
       with open('./new_tianlongbabu.txt', 'a', encoding='utf-8') as ff:
           ff.write(' '.join(seg_list)) # 词汇用空格分开

In [4]:
from gensim.models import word2vec

# 加载语料
sentences = word2vec.Text8Corpus('./new_tianlongbabu.txt')

# 训练模型
model = word2vec.Word2Vec(sentences)

# 选出最相似的10个词
for e in model.most_similar(positive=['乔峰'], topn=10):
   print(e[0], e[1])

萧峰 0.8684546947479248
乌老大 0.8470619320869446
段正淳 0.8451833724975586
游坦之 0.8162813186645508
虚竹 0.8069517612457275
木婉清 0.8062976002693176
苏星河 0.7990739345550537
慕容复 0.7971835136413574
段誉 0.7843468189239502
全冠清 0.7828474044799805


## FastText

word2vec把语料库中的每个单词当成原子的，它会为每个单词生成一个向量。这忽略了单词内部的形态特征，比如：“apple” 和“apples”，这个例子中，两个单词都有较多公共字符，即它们的内部形态类似，但是在传统的word2vec中，这种单词内部形态信息因为它们被转换成不同的id丢失了。为了克服这个问题，fastText使用了字符级别的n-grams来表示一个单词。对于单词“apple”，假设n的取值为3，则它的trigram有“<ap”,  “app”,  “ppl”,  “ple”, “le>”其中，<表示前缀，>表示后缀。于是，我们可以用这些trigram来表示“apple”这个单词，进一步，我们可以用这5个trigram的向量叠加来表示“apple”的词向量。这带来两点好处：1. 对于低频词生成的词向量效果会更好。因为它们的n-gram可以和其它词共享。2. 对于训练词库之外的单词，仍然可以构建它们的词向量。我们可以叠加它们的字符级n-gram向量。

![fasttext](https://ws4.sinaimg.cn/large/006tKfTcly1g117wvdt4ej30go0bg751.jpg)

此架构图没有展示词向量的训练过程。可以看到，和CBOW一样，fastText模型也只有三层：输入层、隐含层、输出层（Hierarchical Softmax），输入都是多个经向量表示的单词，输出都是一个特定的target，隐含层都是对多个词向量的叠加平均。不同的是，CBOW的输入是目标单词的上下文，fastText的输入是多个单词及其n-gram特征，这些特征用来表示单个文档；CBOW的输入单词被onehot编码过，fastText的输入特征是被embedding过；CBOW的输出是目标词汇，fastText的输出是文档对应的类标。值得注意的是，fastText在输入时，将单词的字符级别的n-gram向量作为额外的特征；在输出时，fastText采用了分层Softmax，大大降低了模型训练时间。

In [ ]:
import fastText
import csv
import math
import os
path = "./data/"

def train():
    min_count = [1, 2, 3, 4, 5]
    word_ngrams = [1, 2, 3, 4, 5]
    for x, y in zip(min_count, word_ngrams):
        model = fastText.train_supervised(input="data_label.txt", thread=20, minCount=x, wordNgrams=y,
                                          label="__label__", epoch=10, lr=0.3, loss="hs")
        model.save_model("clf_" + str(x) + "_" + str(y) + ".model")


def get_submission(model):
    question_id = []
    with open("dev_data.txt", "r", encoding="utf-8") as fr:
        for line in fr:
            line = line.split("^")
            question_id.append(line[0])
    model = fastText.load_model(model)
    data = []
    with open("dev_x.txt", "r") as fr:
        for line in fr:
            data.append(line[:-1])
    file_header = ["question_id", "tag@1", "tag@2", "tag@3", "tag@4", "tag@5"]
    csvFile = open("submission.csv", "w")
    writer = csv.writer(csvFile)
    writer.writerow(file_header)
    i = 0
    fr = open("dev_y.txt", "r", encoding="utf-8")
    num_labels = []
    for line in fr:
        line = line.split(" ")
        num_labels.append(len(line))
    for x in data:
        temp = []
        pre = model.predict(x, k=5)
        temp.append(question_id[i])
        for j, label in enumerate(pre[0]):
            if j < num_labels[i]-1:
                temp.append(label[9:])
            else:
                temp.append(-1)
        i += 1
        writer.writerow(temp)


def compute_position_weighted_precision(correct_num, predict_num):
    assert (len(correct_num) == len(predict_num))
    weighted_correct = 0.0
    weighted_predict = 0.0
    for i in range(len(correct_num)):
        weighted_correct += correct_num[i] / math.log(i + 3.0)
        weighted_predict += predict_num[i] / math.log(i + 3.0)
    return weighted_correct / weighted_predict


def compute_recall(correct_num, ground_truth_num):
    return sum(correct_num) / ground_truth_num


def eval(ground_truth_data, predict_data, max_tag_num=5):
    ground_truth = {}
    ground_truth_num = 0.0
    with open(ground_truth_data, 'r', encoding="utf-8") as f:
        lines = csv.reader(f)
        for i, items in enumerate(lines):
            if i == 0:
                continue
            id, true_tag_ids = items[0], items[4]
            ground_truth[id] = true_tag_ids.split('|')
            ground_truth_num += len(ground_truth[id])

    correct_num = [0.0] * max_tag_num
    predict_num = [0.0] * max_tag_num

    with open(predict_data, 'r', encoding="utf-8") as f:
        lines = csv.reader(f)
        for i, items in enumerate(lines):
            if i == 0:
                continue
            # assert (len(items) == max_tag_num + 1)
            id = items[0]
            if id not in ground_truth:
                continue
            true_tag_ids = ground_truth[id]
            for pos, tag_id in enumerate(items[1:]):
                if tag_id == '-1':
                    continue
                predict_num[pos] += 1
                if tag_id in true_tag_ids:
                    correct_num[pos] += 1
    precision = compute_position_weighted_precision(correct_num, predict_num)
    recall = compute_recall(correct_num, ground_truth_num)
    F1 = 2 * precision * recall / (precision + recall)
    print("precision: {}, recall: {}, F1 {}".format(precision, recall, F1))
    return F1


if __name__ == '__main__':
    models = os.listdir(path)
    for model in models:
        print(model + "...")
        get_submission(path + model)
        eval("dev_data.csv", "submission.csv")